In [1]:
import string
import joblib
from os import path

import numpy as np
import pandas as pd
from scipy import sparse
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

from utils import ls, cat

tqdm.pandas()

In [2]:
DATA_DIR = '../../data'
OUTPUT_DIR = '../../data'

ls(DATA_DIR)

DIR      ../../data/stopwords-ru
  2.1 GB ../../data/lenta-ru-news.csv
 20.0  B ../../data/roles.csv
 22.2 MB ../../data/train_lemmas.csv
 22.7 MB ../../data/train_tokens.csv


In [3]:
TRAIN_PATH = f'{DATA_DIR}/lenta-ru-news.csv'
NUM_ROWS = 10000

df_train = pd.read_csv(TRAIN_PATH, nrows=10000) #, index_col='review_id')
display(df_train.head(3))
display(df_train.shape)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17


(10000, 6)

## Features

In [4]:
text_cols = ['title', 'text', 'topic', 'tags']

df_train[text_cols].head()

,title,text,topic,tags
0,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая
1,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая
2,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая
3,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая
4,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая


## Tokenized Text

In [11]:
# !pip install spacy
# python -m spacy download ru_core_news_md

In [5]:
import spacy

nlp = spacy.load('ru_core_news_md', disable=['tagger', 'attribute_ruler', 'senter', 'parser', 'ner'])

In [6]:
train_tokens = df_train[text_cols].fillna('') \
    .progress_applymap(nlp)
    
train_tokens

100%|████████████████████████████████████| 40000/40000 [06:36<00:00, 100.86it/s]


,title,text,topic,tags
0,"(1914, ., Русские, войска, вступили, в, , пре...","(Бои, у, Сопоцкина, и, Друскеник, закончились,...",(Библиотека),"(Первая, мировая)"
1,"(1914, ., Празднование, столетия, М.Ю., Лермон...","(Министерство, народного, просвещения, ,, в, в...",(Библиотека),"(Первая, мировая)"
2,"(1914, ., Das, ist, Nesteroff, !)","(Штабс, -, капитан, П., Н., Нестеров, на, днях...",(Библиотека),"(Первая, мировая)"
3,"(1914, ., Бульдог, -, гонец, под, Льежем)","(Фотограф, -, корреспондент, Daily, Mirror, ра...",(Библиотека),"(Первая, мировая)"
4,"(1914, ., Под, Люблином, пойман, швабский, зверь)","(Лица, ,, приехавшие, в, Варшаву, из, Люблина,...",(Библиотека),"(Первая, мировая)"
...,...,...,...,...
9995,"(На, границе, Дагестана, и, , Чечни, будут, с...","(Глава, МВД, Дагестана, приказал, нарядам, мил...",(Россия),(Все)
9996,"(Путин, пересадит, губернаторов, в, , Госсовет)","(Представители, Кремля, ведут, активный, диало...",(Россия),(Все)
9997,"(В, Москву, съедутся, клоуны, со, , всего, мира)","(В, Москве, 20, июля, открывается, фестиваль, ...",(Россия),(Все)
9998,"(Старшая, Уильямс, выигрывает, чемпионский, ти...","( , Чисто, американским, получился, финал, Уим...",(Спорт),(Все)


In [9]:
TRAIN_LEMMAS_PATH = f'{OUTPUT_DIR}/train_lemmas.csv'

train_tokens \
    .progress_applymap(lambda x: ' '.join(t.lemma_ for t in x)) \
    .to_csv(TRAIN_LEMMAS_PATH)

ls(OUTPUT_DIR, path.basename(TRAIN_LEMMAS_PATH))

100%|██████████████████████████████████| 40000/40000 [00:00<00:00, 50143.24it/s]


 22.2 MB ../../data/train_lemmas.csv


In [10]:
pd.read_csv(TRAIN_LEMMAS_PATH, index_col=[0])

,title,text,topic,tags
0,1914 . русский войско вступить в предел венг...,бой у сопоцкина и друскеник закончиться отступ...,библиотека,первый мировой
1,1914 . празднование столетие м.ю. лермонтов от...,"министерство народный просвещение , в вид прои...",библиотека,первый мировой
2,1914 . das ist nesteroff !,"штабс - капитан п. н. нестеров на день , увиде...",библиотека,первый мировой
3,1914 . бульдог - гонец под льеж,фотограф - корреспондент daily mirror рассказы...,библиотека,первый мировой
4,1914 . под люблин поймать швабский зверь,"лицо , приехать в варшава из люблин , передава...",библиотека,первый мировой
...,...,...,...,...
9995,на граница дагестан и чечня будут стрелять в...,глава мвд дагестан приказать наряд милиция на ...,россия,все
9996,путин пересадит губернатор в госсовет,представитель кремль вести активный диалог с г...,россия,все
9997,в москва съедутся клоун со весь мир,"в москва 20 июль открываться фестиваль "" плане...",россия,все
9998,старший уильямс выигрывать чемпионский титул,чисто американский получиться финал уимблдон...,спорт,все
